In [9]:
%pip install statsmodels

import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.filterwarnings('ignore')


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
df = pd.read_csv("/workspaces/TSCDIA-/proyecto_integrador/tp/brazilian_ecommerce.csv")

df.head()

df.info()

df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119143 entries, 0 to 119142
Data columns (total 37 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       119143 non-null  object 
 1   customer_id                    119143 non-null  object 
 2   order_status                   119143 non-null  object 
 3   order_purchase_timestamp       119143 non-null  object 
 4   order_approved_at              118966 non-null  object 
 5   order_delivered_carrier_date   117057 non-null  object 
 6   order_delivered_customer_date  115722 non-null  object 
 7   order_estimated_delivery_date  119143 non-null  object 
 8   order_item_id                  118310 non-null  float64
 9   product_id                     118310 non-null  object 
 10  seller_id                      118310 non-null  object 
 11  shipping_limit_date            118310 non-null  object 
 12  price                         

,order_item_id,price,freight_value,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,review_score,customer_zip_code_prefix,payment_sequential,payment_installments,payment_value
count,118310.000000,118310.000000,118310.000000,116601.000000,116601.000000,116601.000000,118290.000000,118290.000000,118290.000000,118290.000000,118146.000000,119143.000000,119140.000000,119140.000000,119140.000000
mean,1.196543,120.646603,20.032387,48.767498,785.967822,2.205161,2112.250740,30.265145,16.619706,23.074799,4.015582,35033.451298,1.094737,2.941246,172.735135
std,0.699489,184.109691,15.836850,10.033540,652.584121,1.717452,3786.695111,16.189367,13.453584,11.749139,1.400436,29823.198969,0.730141,2.777848,267.776077
min,1.000000,0.850000,0.000000,5.000000,4.000000,1.000000,0.000000,7.000000,2.000000,6.000000,1.000000,1003.000000,1.000000,0.000000,0.000000
25%,1.000000,39.900000,13.080000,42.000000,346.000000,1.000000,300.000000,18.000000,8.000000,15.000000,4.000000,11250.000000,1.000000,1.000000,60.850000
50%,1.000000,74.900000,16.280000,52.000000,600.000000,1.000000,700.000000,25.000000,13.000000,20.000000,5.000000,24240.000000,1.000000,2.000000,108.160000
75%,1.000000,134.900000,21.180000,57.000000,983.000000,3.000000,1800.000000,38.000000,20.000000,30.000000,5.000000,58475.000000,1.000000,4.000000,189.240000
max,21.000000,6735.000000,409.680000,76.000000,3992.000000,20.000000,40425.000000,105.000000,105.000000,118.000000,5.000000,99990.000000,29.000000,24.000000,13664.080000


In [8]:
for col in df.columns:
    valores_unicos = df[col].unique()
    print(f"Columna '{col}': {len(valores_unicos)} valores únicos")
    print(f"Valores: {valores_unicos}")
    print("-" * 80)

Columna 'order_id': 99441 valores únicos
Valores: ['e481f51cbdc54678b7cc49136f2d6af7' '53cdb2fc8bc7dce0b6741e2150273451'
 '47770eb9100c2d0c44946d9cf07ec65d' ... '83c1379a015df1e13d02aae0204711ab'
 '11c177c8e97725db2631073c19f07b62' '66dea50a8b16d9b4dee7af250b4be1a5']
--------------------------------------------------------------------------------
Columna 'customer_id': 99441 valores únicos
Valores: ['9ef432eb6251297304e76186b10a928d' 'b0830fb4747a6c6d20dea0b8c802d7ef'
 '41ce2a54c0b03bf3443c3d931a367089' ... '1aa71eb042121263aafbe80c1b562c9c'
 'b331b74b18dc79bcdf6532d51e1637c1' 'edb027a75a1449115f6b43211ae02a24']
--------------------------------------------------------------------------------
Columna 'order_status': 8 valores únicos
Valores: ['delivered' 'invoiced' 'shipped' 'processing' 'unavailable' 'canceled'
 'created' 'approved']
--------------------------------------------------------------------------------
Columna 'order_purchase_timestamp': 98875 valores únicos
Valores: ['2017-

In [13]:
def predict_next_sale(product_id, sales_data): # Filtrar datos para el producto específico 
    product_sales = sales_data[sales_data['product_id'] == product_id].set_index('order_purchase_timestamp')['sales']
    if len(product_sales) < 10:  # Necesita al menos 10 puntos de datos para un modelo confiable
        return f"Datos insuficientes para {product_id} (solo {len(product_sales)} ventas)"
    # Verificar estacionariedad con Dickey-Fuller
    try:
        result = adfuller(product_sales)
        if result[1] > 0.05:  # Si no es estacionaria, diferenciar
            product_sales = product_sales.diff().dropna()
    except:
        pass

# Ajustar modelo SARIMA (parámetros básicos: p=1, d=1, q=1, P=1, D=1, Q=1, s=7 para estacionalidad semanal)
    try:
        model = SARIMAX(product_sales, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7))
        model_fit = model.fit(disp=False)
    
    # Pronosticar 1 paso adelante (próxima fecha)
        forecast = model_fit.forecast(steps=1) # type: ignore
        next_sale_date = pd.to_datetime(product_sales.index[-1]) + pd.Timedelta(days=1)
    
        return f"Producto {product_id}: Próxima venta el {next_sale_date.date()} con {forecast.iloc[0]:.2f} unidades"
    except Exception as e:
        return f"Error en modelo para {product_id}: {str(e)}"

unique_products = df['product_id'].dropna().unique() 
predictions = {}

sales_per_product = df.groupby(['product_id', 'order_purchase_timestamp']).size().reset_index(name='sales')

for pid in unique_products[:5]: predictions[pid] = predict_next_sale(pid, sales_per_product)

for pid, pred in predictions.items(): print(pred)

Datos insuficientes para 87285b34884572647811a353c7ac498a (solo 4 ventas)
Producto 595fac2a385ac33a80bd5114aec74eb8: Próxima venta el 2018-08-24 con 1.26 unidades
Datos insuficientes para aa4383b373c6aca5d8797843e5594415 (solo 3 ventas)
Datos insuficientes para d0b61bfb1de832b15ba9d266ca96e5b0 (solo 4 ventas)
Producto 65266b2da20d04dbe00c5c2d3bb7859e: Próxima venta el 2018-07-25 con 1.05 unidades
